In [1]:
%cd "/kaggle/working"

/kaggle/working


In [2]:
# !sudo apt-get update

In [3]:
# !echo 'debconf debconf/frontend select Noninteractive' | sudo debconf-set-selections
# !sudo apt-get install -y -q cuda

# Installation

In [4]:
!/usr/local/cuda/bin/nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2021 NVIDIA Corporation
Built on Mon_May__3_19:15:13_PDT_2021
Cuda compilation tools, release 11.3, V11.3.109
Build cuda_11.3.r11.3/compiler.29920130_0


In [5]:
!pip install --upgrade git+https://github.com/frehseg/nvcc4jupyter.git

  Cloning https://github.com/frehseg/nvcc4jupyter.git to /tmp/pip-req-build-v2ke3p0f
  Running command git clone --filter=blob:none --quiet https://github.com/frehseg/nvcc4jupyter.git /tmp/pip-req-build-v2ke3p0f
  Resolved https://github.com/frehseg/nvcc4jupyter.git to commit a599751b98cbb8537fe9609198030b55669535da
  Preparing metadata (setup.py) ... - \ done
  Created wheel for NVCCPlugin: filename=NVCCPlugin-0.2-py3-none-any.whl size=2207 sha256=307650e68eab7242c3e4b330a0e063ccddc5adf98dbd9899354beabfc1e3c72d
  Stored in directory: /tmp/pip-ephem-wheel-cache-au_vpypi/wheels/4c/02/de/02070cb0bea5e90c8e1412113a24c8967f93b63f8806e2b398
Successfully built NVCCPlugin


In [6]:
%load_ext nvcc_plugin

In [7]:
import os
os.environ['PATH'].split(';')

['/opt/bin:/opt/conda/bin:/usr/local/nvidia/bin:/usr/local/cuda/bin:/usr/local/sbin:/usr/local/bin:/usr/sbin:/usr/bin:/sbin:/bin']

# Code

## CUDA Utilities

In [8]:
%%writefile cuda_stuff.cuh
#ifndef cuda_stuff_H
#define cuda_stuff_H

#include <stdio.h>
#include <stdlib.h>
#include <cuda.h>
#include <cuda_runtime.h>

//MACRO TO DEBUG CUDA FUNCTIONS
/** Error checking,
 *  taken from https://stackoverflow.com/questions/14038589/what-is-the-canonical-way-to-check-for-errors-using-the-cuda-runtime-api
 */
#define gpuErrchk(ans) { gpuAssert((ans), __FILE__, __LINE__); }
inline void gpuAssert(cudaError_t code, const char *file, int line, bool abort=true)
{
   if (code != cudaSuccess) 
   {
      fprintf(stderr,"GPUassert: %s %s %d\n", cudaGetErrorString(code), file, line);
      if (abort) exit(code);
   }
}

void device_synchronize();

#endif


Writing cuda_stuff.cuh


In [9]:
%%writefile cuda_stuff.cu
#include <stdio.h>
#include <stdlib.h>
#include <cuda.h>
#include <cuda_runtime.h>

#include "cuda_stuff.cuh"

void device_synchronize(){
    gpuErrchk(cudaDeviceSynchronize());
}

Writing cuda_stuff.cu


## Matrix Tools

In [10]:
%%writefile fmatrix.cuh
#ifndef fmatrices_H
#define fmatrices_H
#include <stddef.h> 

typedef struct {
    float* data;
    size_t cols;
    size_t rows;
} fmatrix;

/* transform matrix index to vector offset
   Since CUDA uses column major, 
   nb_rows = number of rows */
#define IDX2C(i,j,nb_rows) (((j)*(nb_rows))+(i))

/* Access element (i,j) of matrix mat */
#define getfm(mat,i,j) (mat.data[IDX2C(i,j,mat.rows)])


size_t fmatrix_elements(fmatrix mat);
size_t fmatrix_size(fmatrix mat);
void fmatrix_init(fmatrix mat, float f);
/** Assert that the matrix is coherent: all fields nonzero. */
void fmatrix_assert();

fmatrix fmatrix_create_on_host(size_t rows, size_t cols);
fmatrix fmatrix_create_on_device(size_t rows, size_t cols);

void fmatrix_data_to_host(fmatrix mat_host, fmatrix mat_device);
void fmatrix_data_to_device(fmatrix mat_host, fmatrix mat_device);

void fmatrix_free_on_host(fmatrix* mat);
void fmatrix_free_on_device(fmatrix* mat);

/** Print the first nb rows of the matrix mat
 *  on the host. 
 *  If nb<0, print all rows. 
 */
void fmatrix_host_print(fmatrix mat, int nb=-1);

/** Print the first nb rows of the matrix mat
 *  on the device. 
 *  If nb<0, print all rows. 
 */
void fmatrix_device_print(fmatrix mat, int nb=-1);

#endif


Writing fmatrix.cuh


In [11]:
%%writefile fmatrix.cu
#include <assert.h>
#include <stdio.h>
#include <stdlib.h>
#include <cuda.h>
#include <cuda_runtime.h>

#include "cuda_stuff.cuh"
#include "fmatrix.cuh"

size_t fmatrix_elements(fmatrix mat) {
     return mat.cols*mat.rows;
}

size_t fmatrix_size(fmatrix mat) {
     return fmatrix_elements(mat) * sizeof(mat.data[0]);
}

void fmatrix_init(fmatrix mat, float f) {
    for (int i = 0; i < mat.rows; i++){
        for (int j = 0; j < mat.cols; j++){
            mat.data[IDX2C(i,j,mat.rows)] = f; 
    }
  }
} 

void fmatrix_assert(fmatrix mat) {
    assert(mat.data);
    assert(mat.cols);
    assert(mat.rows);
}



fmatrix fmatrix_create_on_host(size_t rows, size_t cols) {
    assert(cols>0);
    assert(rows>0);
    fmatrix mat;
    mat.cols = cols;
    mat.rows = rows;
    mat.data = (float*)malloc(fmatrix_size(mat)); 
    assert(mat.data);
    return mat;
}

fmatrix fmatrix_create_on_device(size_t rows, size_t cols) {
    assert(cols>0);
    assert(rows>0);
    fmatrix mat;
    mat.cols = cols;
    mat.rows = rows;
    gpuErrchk( 
        cudaMalloc((void **)&(mat.data), fmatrix_size(mat)) 
    );
    return mat;
}

void fmatrix_data_to_device(fmatrix mat_host, fmatrix mat_device) {
    fmatrix_assert(mat_host);
    fmatrix_assert(mat_device);
    assert(mat_host.cols==mat_device.cols);
    assert(mat_host.rows==mat_device.rows);
    gpuErrchk( 
        cudaMemcpy( mat_device.data, mat_host.data, 
                   fmatrix_size(mat_host), 
                   cudaMemcpyHostToDevice 
                   )
        );
}

void fmatrix_data_to_host(fmatrix mat_host, fmatrix mat_device) {
    fmatrix_assert(mat_host);
    fmatrix_assert(mat_device);
    assert(mat_host.cols==mat_device.cols);
    assert(mat_host.rows==mat_device.rows);
    gpuErrchk(
        cudaMemcpy( mat_host.data, mat_device.data,  
                   fmatrix_size(mat_device), 
                   cudaMemcpyDeviceToHost 
                   )
        );
}

void fmatrix_free_on_host(fmatrix* mat) {
    fmatrix_assert(*mat);  
  free(mat->data);
  mat->data = 0;
  mat->cols = 0;
  mat->rows = 0;
}

void fmatrix_free_on_device(fmatrix* mat) {
    fmatrix_assert(*mat);  
  gpuErrchk(cudaFree(mat->data));
  mat->data = 0;
  mat->cols = 0;
  mat->rows = 0;
}

void fmatrix_host_print(fmatrix mat, int nb){
    if (nb<0 || nb > mat.rows) {
        nb = mat.rows;
    }
    printf("[\n");
    for (int i = 0 ; i < nb; i++){
      for (int j = 0 ; j<mat.cols; j++){
        printf("%f", getfm(mat,i,j));
        if (j+1<mat.cols) {
          printf(",\t");
        }
      }
      if (i+1<nb) {
        printf(";\n");
      }
    }
    if (nb < mat.rows) {
      printf("\n...\n");
    }
  printf("\n]\n");
}

void fmatrix_device_print(fmatrix mat, int nb){
   // allocate copy
   fmatrix tmp = fmatrix_create_on_host(mat.rows, mat.cols);
   fmatrix_data_to_host(tmp, mat);
   fmatrix_host_print(tmp,nb);
   fmatrix_free_on_host(&tmp);
}



Writing fmatrix.cu


## Matrix Math

In [12]:
%%writefile sgemm.cuh
#ifndef sgemm_H
#define sgemm_H

#include <string>
#include "fmatrix.cuh"

void mat_mul(fmatrix A, fmatrix B, fmatrix C, std::string arg);

#endif

Writing sgemm.cuh


In [13]:
%%writefile sgemm.cu
#include <stdio.h>
#include <stdlib.h>
#include <string>
#include <time.h>
#include <math.h>
#include <cuda.h>
#include <cuda_runtime.h>
#include "cublas_v2.h"

#include "cuda_stuff.cuh"
#include "sgemm.cuh"
#include "fmatrix.cuh"

#define THREADS_PER_BLOCK 1024
#define TILE_WIDTH 32

using namespace std;

/* basic matrix multiplication C = alpha*A*B + beta*C on host as reference for the speedup */
void matrixMultiplication_basic_host(float alpha, fmatrix A, fmatrix B, float beta, fmatrix C) { 
  float tmp = 0;
  for (int i = 0; i<A.rows; i++){
    for (int j = 0; j<B.cols; j++){
      for (int k = 0; k<A.cols; k++){
        tmp += alpha * getfm(A,i, k) * getfm(B, k, j);
      }
      C.data[IDX2C(i,j,C.rows)] = beta * C.data[IDX2C(i,j,C.rows)] + tmp; 
      tmp = 0;
    }
  } 
}
               
/* 3 different versions of matrix multiplication C = alpha*A*B + beta*C on device */
__global__
void matmul_basic_kernel(float *A, float *B, float *C, int nb_ColA, int nb_ColB, int nb_LigneA, int nb_LigneB) {
 float tmp = 0;
 int row = blockIdx.x;
 int col = threadIdx.x;
  for (int i = 0; i < nb_ColA; i++) {
      tmp += A[i * nb_LigneA + row] * B[col * nb_LigneB + i];
 }
 C[col * nb_LigneA + row] = tmp;
}
void matrixMultiplication_basic(float alpha, fmatrix d_A, fmatrix d_B, float beta, fmatrix d_C) { 
  // declaration of dimGrid and dimBlock
  int dimGrid=d_A.rows;
  int dimBlock=d_A.rows;
  
  matmul_basic_kernel <<< dimGrid, dimBlock >>> (d_A.data, d_B.data, d_C.data, d_A.cols, d_B.cols, d_A.rows, d_B.rows);

} 

/**********************/
__global__
void matmul_tiled_kernel(float *A, float *B, float *C, int nb_ColA, int nb_ColB, int nb_LigneA, int nb_LigneB){
  const int tile_width=16;
  int N=nb_LigneA/tile_width;
  int i=threadIdx.x/tile_width;
  int j=threadIdx.x-i*tile_width;
  int row=blockIdx.x/N;
  int col=blockIdx.x-row*N;
  __shared__ int sharedA[tile_width*tile_width];
  __shared__ int sharedB[tile_width*tile_width];
  for (int n = 0; n < N; n++){
      sharedB[j*tile_width+i]=B[(j+tile_width*col)*nb_LigneB+(i+tile_width*n)];
      sharedA[j*tile_width+i]=A[(j+tile_width*n)*nb_LigneA+(i+tile_width*row)];
      __syncthreads();
      float tmp=0;
      for (int k = 0; k < tile_width; k++){
          tmp += sharedA[k*tile_width+i]*sharedB[j*tile_width+k];
      }
      C[(j+tile_width*col)*nb_LigneA+(i+tile_width*row)]=C[(j+tile_width*col)*nb_LigneA+(i+tile_width*row)]+tmp;
  }
}


void matrixMultiplication_tiled(float alpha, fmatrix d_A, fmatrix d_B, float beta, fmatrix d_C){
  // declaration of dimGrid and dimBlock
  const int tile_width=16;
  int N=d_A.rows/tile_width;
  int dimGrid=N*N;
  int dimBlock=tile_width*tile_width;
  matmul_tiled_kernel <<< dimGrid, dimBlock >>> (d_A.data, d_B.data, d_C.data, d_A.cols, d_B.cols, d_A.rows, d_B.rows);
}

/**********************/
void matrixMultiplication_cublas(float alpha, fmatrix d_A, fmatrix d_B, float beta, fmatrix d_C){
  cublasHandle_t handle;
  cublasCreate(&handle);
  int m=d_A.rows;
  int n=d_B.cols;
  int k=d_A.cols;
  cublasSgemm(handle, CUBLAS_OP_N, CUBLAS_OP_N, m, n, k, &alpha, d_A.data, m, d_B.data, k, &beta, d_C.data, m);
  cublasDestroy(handle);
}



/*MAIN SGEMM*/
void gen_mat_mul(float alpha, fmatrix A, fmatrix B, float beta, fmatrix C, std::string arg){
    if (arg == "cpu"){
        matrixMultiplication_basic_host(alpha, A, B, beta, C);
    } else {
      /* kernel function*/  
      if (arg == "gpu_basic"){
          matrixMultiplication_basic(alpha, A, B, beta, C);
      
      } else if (arg == "gpu_tiled"){
          matrixMultiplication_tiled(alpha, A, B, beta, C);
      
      } else if (arg == "gpu_cublas"){
         matrixMultiplication_cublas(alpha, A, B, beta, C);
    
      } else{
          printf("Matrix Multiplication argument is Wrong");
          exit(0);
      }
      // wait for everything to finish
      printf("Synchro\n");
      device_synchronize();
    }
}

void mat_mul(fmatrix A, fmatrix B, fmatrix C, std::string arg){
 gen_mat_mul(1.0, A, B, 0.1, C, arg);   
}


Writing sgemm.cu


# Main

In [14]:
%%writefile main.cu

#include <stdio.h>
#include <stdlib.h>
#include "fmatrix.cuh"
#include "sgemm.cuh"

#define TILE_WIDTH 32
#define SIZE 20

int main(void)
{
  /* Allocate and initialize data on host */
  fmatrix A = fmatrix_create_on_host(TILE_WIDTH * SIZE, TILE_WIDTH * SIZE);
  fmatrix_init(A, 1.0);    
  fmatrix B = fmatrix_create_on_host(TILE_WIDTH * SIZE, TILE_WIDTH * SIZE);
  fmatrix_init(B, 2.0);
  fmatrix C = fmatrix_create_on_host(TILE_WIDTH * SIZE, TILE_WIDTH * SIZE);
  fmatrix_init(C, 0.0);

  /* Allocate data on device */
  fmatrix d_A = fmatrix_create_on_device(TILE_WIDTH * SIZE, TILE_WIDTH * SIZE);
  fmatrix d_B = fmatrix_create_on_device(TILE_WIDTH * SIZE, TILE_WIDTH * SIZE);
  fmatrix d_C = fmatrix_create_on_device(TILE_WIDTH * SIZE, TILE_WIDTH * SIZE);
  
  /* Transfer A and B on device */
  fmatrix_data_to_device(A, d_A);
  fmatrix_data_to_device(B, d_B); 
  fmatrix_data_to_device(C, d_C);

  clock_t start, end;
  float cpu_time_used;   

  /* Start calculation "cpu", "gpu_basic", "gpu_tiled", "gpu_cublas" */ 
  /************** "cpu" *******************/
  start = clock();
  mat_mul(A, B, C, "cpu");
  end = clock();
  cpu_time_used = ((double) (end - start)) * 1000 / CLOCKS_PER_SEC;
  printf("Time taken by CPU in milliseconds: %.2f\n", cpu_time_used);
 
  
  /* Result correctness */
  {
    float maxError = 0.0f;
    for (int i = 0; i < TILE_WIDTH * SIZE; i++){
      for (int j = 0; j < TILE_WIDTH * SIZE; j++){
        maxError = max(maxError, abs(getfm(C,i,j)- 2*TILE_WIDTH * SIZE));
      }
    } 
    printf("Max error: %f\n", maxError);
  }
  fmatrix_init(C, 0.0); 

  /************** "gpu_basic" *******************/
  start = clock();
  mat_mul(d_A, d_B, d_C, "gpu_basic");
  end = clock();
  cpu_time_used = ((double) (end - start)) * 1000 / CLOCKS_PER_SEC;
  printf("GPU basic matrix multiplication in milliseconcs : %.2f\n", cpu_time_used);
 
  /* Retrieve the result */
  fmatrix_data_to_host(C, d_C);
  /* Result correctness */
  {
    float maxError = 0.0f;
    for (int i = 0; i < TILE_WIDTH * SIZE; i++){
      for (int j = 0; j < TILE_WIDTH * SIZE; j++){
        maxError = max(maxError, abs(getfm(C,i,j)- 2*TILE_WIDTH * SIZE));
      }
    }   
    printf("Max error: %f\n", maxError);
  } 
  fmatrix_init(C, 0.0); 
  fmatrix_data_to_device(C, d_C);
 

 /************** "gpu_tiled" *******************/
  start = clock();
  mat_mul(d_A, d_B, d_C, "gpu_tiled");
  end = clock();
  cpu_time_used = ((double) (end - start)) * 1000 / CLOCKS_PER_SEC;
  printf("GPU tiled matrix multiplication in milliseconcs : %.2f\n", cpu_time_used);
 
  /* Retrieve the result */
  fmatrix_data_to_host(C, d_C);
  /* Result correctness */
  {
    float maxError = 0.0f;
    for (int i = 0; i < TILE_WIDTH * SIZE; i++){
      for (int j = 0; j < TILE_WIDTH * SIZE; j++){
        maxError = max(maxError, abs(getfm(C,i,j)- 2*TILE_WIDTH * SIZE));
      }
    }   
    printf("Max error: %f\n", maxError);
  } 
  fmatrix_init(C, 0.0); 
  fmatrix_data_to_device(C, d_C);


  /************** "gpu_cublas" *******************/
  start = clock();
  mat_mul(d_A, d_B, d_C, "gpu_cublas");
  end = clock();
 
  cpu_time_used = ((double) (end - start)) * 1000 / CLOCKS_PER_SEC;
  printf("GPU cuBLAS matrix multiplication in milliseconcs : %.2f\n", cpu_time_used);
 
  /* Retrieve the result */
  fmatrix_data_to_host(C, d_C);
  /* Result correctness */
  {
    float maxError = 0.0f;
    for (int i = 0; i < TILE_WIDTH * SIZE; i++){
      for (int j = 0; j < TILE_WIDTH * SIZE; j++){
        maxError = max(maxError, abs(getfm(C,i,j)- 2*TILE_WIDTH * SIZE));
      }
    }   
    printf("Max error: %f\n", maxError);
  } 
  fmatrix_init(C, 0.0); 
  fmatrix_data_to_device(C, d_C);

  /* Free */ 
  fmatrix_free_on_host(&A);
  fmatrix_free_on_host(&B);
  fmatrix_free_on_host(&C);
  fmatrix_free_on_device(&d_A);
  fmatrix_free_on_device(&d_B);
  fmatrix_free_on_device(&d_C);
}

Writing main.cu


# Compiling

In [15]:
!/usr/local/cuda/bin/nvcc -lcublas sgemm.cu fmatrix.cu cuda_stuff.cu main.cu

# Experiments

In [16]:
! ./a.out

Time taken by CPU in milliseconds: 1068.97
Max error: 0.000000
Synchro
GPU basic matrix multiplication in milliseconcs : 7.62
Max error: 0.000000
Synchro
GPU tiled matrix multiplication in milliseconcs : 1.51
Max error: 0.000000
Synchro
GPU cuBLAS matrix multiplication in milliseconcs : 437.20
Max error: 0.000000


In [17]:
!compute-sanitizer ./a.out "UNIT"

========= COMPUTE-SANITIZER
========= Error: Detected an unsupported driver version. Please update your CUDA installation to 12.0 or newer (driver version 525 or higher)
Time taken by CPU in milliseconds: 1243.08
Max error: 0.000000
Synchro
GPU basic matrix multiplication in milliseconcs : 7.61
Max error: 0.000000
Synchro
GPU tiled matrix multiplication in milliseconcs : 1.40
Max error: 0.000000
Synchro
GPU cuBLAS matrix multiplication in milliseconcs : 360.63
Max error: 0.000000
========= ERROR SUMMARY: 1 error


# Results

Over 10 executions, the mean execution time obtained was :

*   CPU : 1078 ms
*   GPU basic : 23.94 ms
*   GPU tiled : 10.74 ms (TILE_WIDTH=32) et 7.19 ms (TILE_WIDTH=16)
*   GPU cublas : 319 ms

The matrix multiplication is more efficient with GPU tiled configuration, and is even faster by diminushing TILE_WIDTH value